### Introducción a los Datos Masivos y a la Ciencia en Abierto

Partiendo del ejemplo realizado con el método de la bisección, obtener las tres raíces del polinomio considerado en esa sesión (x^3-3x^2-2x+6) con el método de la bisección y el Newton-Raphson, y comparar las soluciones obtenidas considerando el umbral 1e-5.

Escribir un fichero de texto plano con las soluciones obtenidas así como la precisión y el número de pasos realizados para obtener cada una de las soluciones.

Se entregará un script de R, un archivo Rmd o un notebook de Jupyter con el código empleado para resolver la tarea, así como el fichero generado. El documento debe incluir las explicaciones pertinentes de los pasos considerados relevantes/esenciales para la resolución de la tarea.

---

# Root-finding overview: Bisection vs Newton–Raphson
### Author: Claudia Vello

**Objective.** We will find the three real roots of the polynomial
$ f(x) = x^3 - 3x^2 - 2x + 6 $
using two classic numerical methods—**Bisection** and **Newton–Raphson**—and compare the results at tolerance $ \text{tol} = 10^{-5} $.

---

## Bisection method

**Idea.** If $f(a)$ and $f(b)$ have opposite signs, by the Intermediate Value Theorem there is at least one root in $[a,b]$. Bisection repeatedly halves the interval, keeping the subinterval where the sign change persists.

**Algorithm.**
1. Choose $a,b$ with $f(a)f(b) < 0$.
2. Let $m = (a+b)/2$.
3. If $f(a)f(m) \le 0$, set $b \leftarrow m$ else $a \leftarrow m$.
4. Repeat until a stopping criterion is met.

**Convergence.** Linear. After $k$ steps the interval length is $\frac{b-a}{2^k}$; a rigorous **error bound** on the root location is $\le \frac{b-a}{2^{k+1}}$.

**Pros**
- Guaranteed to converge if the initial bracket has a sign change.
- Provides a clean a-priori error bound from the interval length.

**Cons.**
- Converges slower than Newton (linear vs quadratic).

---

## Newton–Raphson method

**Idea.** Starting from $x_0$, approximate $f$ by its tangent and step to the x-intercept:
$$
x_{k+1} \;=\; x_k - \frac{f(x_k)}{f'(x_k)}.
$$

**Convergence.** Quadratic if the start is close to a simple root and $f'(x^*) \neq 0$. Far from a root—or near points where $f'(x)\approx 0$—the method can fail or diverge.

**Pros**
- Very fast near the solution (quadratic convergence).

**Cons.**
- Requires a good initial guess and a non-negligible derivative.
- No built-in bracketing; can jump away or oscillate.

---

## Stopping criteria used in this notebook

We enforce safeguards that reflect **both** positional accuracy and equation satisfaction:

- **Bisection:** stop when either
  $$
  |f(m)|<\text{tol} \text{   or   } \max\!\Big(\tfrac{b-a}{2},\, |f(m)|\Big) < \text{tol}.
  $$
  This demands a *tight bracket* and a *small residual*.

- **Newton–Raphson:** stop when  
  $$
  \min\!\big(|\Delta x|,\, |f(x)|\big) < \text{tol},
  $$ 
  where $|\Delta x| = |x_{k+1}-x_k|$. Either a tiny step or a tiny residual is acceptable in practice.

**Iteration cap.** We also set a generous maximum number of iterations to prevent infinite loops in pathological cases.

---

## Numerical safeguards and practical choices

- **Bracketing for Bisection:** We will pick intervals $[a,b]$ where a sign change is verified by sampling $f$ at a few points.
- **Initial guesses for Newton:** We will choose $x_0$ values suggested by the shape/signs of $f$ so that Newton starts near each root.
- **Derivative checks:** If $|f'(x)|$ is too small, Newton steps are unsafe; we will detect this and stop with a warning.
- **Error reporting:**  
  - Bisection: report $(b-a)/2$ as a rigorous bound on the root location.  
  - Newton: report the last step size $|\Delta x|$ as a practical error proxy.

---

## Output

For each root and each method we will report:
- the **estimated root**,
- the **residual** $|f(x^*)|$,
- an **error estimate** (interval half-width for Bisection; last step size for Newton),
- and the **number of iterations**.

We will also write a **plain-text file** with these results to include in the submission.

In [14]:
# Define function and derivative
f  <- function(x) x^3 - 3 * x^2 - 2 * x + 6
df <- function(x) 3 * x^2 - 6 * x - 2

# Tolerance and iteration limits
tol   <- 1e-5 # required tolerance
maxit <- 1000  # bisection needs ~ O(log2((b-a)/tol)), prevents infinite loops

In [15]:
# Bisection method
bisection <- function(f, a, b, tol, maxit) {
  fa <- f(a); fb <- f(b)
  if (fa * fb > 0) stop("Bisection: f(a) and f(b) must have opposite signs.")
  it <- 0 # iteration counter
  while (it < maxit) {
    it <- it + 1
    m  <- (a + b)/2 # midpoint
    fm <- f(m) # evaluation at midpoint

    # If midpoint is (numerically) a root, return immediately
    if (abs(fm) < tol) {
      return(list(root = m,
                  iterations = it,
                  residual = abs(fm),
                  error_bound = (b - a)/2))
    }

    # If the bracket is already tight enough, return
    if ((b - a)/2 < tol) {
      return(list(root = m,
                  iterations = it,
                  residual = abs(fm),
                  error_bound = (b - a)/2))
    }

    # Update bracket, keep zero on the bracketed side (<= retains exact-zero)
    if (fa * fm <= 0) { # <= ensures m remains inside the retained half, we don’t “lose” the root by moving the bracket to the wrong side
      b  <- m; fb <- fm
    } else {
      a  <- m; fa <- fm
    }
  }

  warning("Bisection: reached maxit without convergence.")
  m <- (a + b)/2
  list(root = m, iterations = it, residual = abs(f(m)), error_bound = (b - a)/2)
}

In [16]:
# Newton–Raphson method
newton_raphson <- function(f, df, x0, tol, maxit) {
  x <- x0
  it <- 0
  last_dx <- NA
  fx_new <- f(x)  # initialize residual at start (in case we break early)

  for (k in 1:maxit) {
    it <- k
    dfx <- df(x)
    if (!is.finite(dfx) || abs(dfx) < .Machine$double.eps^0.5) {
      warning(sprintf("Newton: near-zero or non-finite derivative at x=%.6f. Stopping.", x))
      break
    }
    fx <- f(x)
    dx <- - fx / dfx
    x  <- x + dx
    last_dx <- abs(dx)

    fx_new <- f(x)  # residual after the update

    if (min(abs(fx_new), last_dx) < tol) {
      return(list(root = x,
                  iterations = it,
                  residual = abs(fx_new),
                  step_last = last_dx))
    }
  }

  warning("Newton: reached maxit or stopped early without convergence.")
  list(root = x, iterations = it, residual = abs(fx_new), step_last = last_dx)
}

In [17]:
bis_suggestions    <- list(c(-2, -1), c(1,  2), c(2.5, 3.5))

# Ask for [a,b] with a suggested default, accept Enter to take the suggestion
get_bisection_interval_suggested <- function(f, suggestion_ab) {
  repeat {
    cat(sprintf("\nEnter interval [a,b] for bisection (press Enter to accept the suggestion):\n"))
    cat(sprintf("Suggested: [a, b] = [%.4f, %.4f]\n", suggestion_ab[1], suggestion_ab[2]))

    # Read 'a' with default
    a_in <- readline(sprintf("a = [default %.4f]: ", suggestion_ab[1]))
    if (a_in == "") {
      a <- suggestion_ab[1]
    } else {
      a <- suppressWarnings(as.numeric(a_in))
    }

    # Read 'b' with default
    b_in <- readline(sprintf("b = [default %.4f]: ", suggestion_ab[2]))
    if (b_in == "") {
      b <- suggestion_ab[2]
    } else {
      b <- suppressWarnings(as.numeric(b_in))
    }

    # Validate numeric input
    if (is.na(a) || is.na(b)) {
      cat("Please enter numeric values (or press Enter to accept the defaults).\n")
      next
    }
    # Validate order
    if (a >= b) {
      cat("Condition a < b is required.\n")
      next
    }
    # Validate sign change
    if (f(a) * f(b) > 0) {
      cat("f(a) and f(b) have the same sign. No root is guaranteed in [a,b]. Try again.\n")
      next
    }

    cat(sprintf("Interval accepted: [%.4f, %.4f]\n", a, b))
    return(c(a, b))
  }
}

In [18]:
newton_suggestions <- c(-1.5, 1.5, 3.1)

# Ask for x0 with a suggested default, accept Enter to take the suggestion
get_newton_start_suggested <- function(f, df, suggestion_x0) {
  repeat {
    cat(sprintf("\nEnter initial guess for Newton–Raphson (press Enter to accept the suggestion):\n"))
    cat(sprintf("Suggested: x0 = %.4f\n", suggestion_x0))

    x0_in <- readline(sprintf("x0 = [default %.4f]: ", suggestion_x0))
    if (x0_in == "") {
      x0 <- suggestion_x0
    } else {
      x0 <- suppressWarnings(as.numeric(x0_in))
    }

    # Validate numeric input
    if (is.na(x0)) {
      cat("Please enter a numeric value (or press Enter to accept the default).\n")
      next
    }
    # Guard against near-zero derivative
    if (abs(df(x0)) < .Machine$double.eps^0.5) {
      cat("Derivative near zero at this x0; choose another value.\n")
      next
    }

    cat(sprintf("Starting Newton from x0 = %.4f (f(x0)=%.4f, f'(x0)=%.4f)\n", x0, f(x0), df(x0)))
    return(x0)
  }
}

In [19]:
# Run both methods and collect results
n_roots <- 3  # compute three roots for each method, with suggestions

# Bisection results
res_bis <- lapply(seq_len(n_roots), function(i){
  cat(sprintf("\n[Bisection %d/%d]\n", i, n_roots))
  ab  <- get_bisection_interval_suggested(f, bis_suggestions[[i]])
  out <- bisection(f, ab[1], ab[2], tol = tol, maxit = maxit)
  c(method       = "Bisection",
    root_id      = i,
    init         = sprintf("[%.4f, %.4f]", ab[1], ab[2]),
    estimate     = out$root,
    residual     = out$residual,
    err_estimate = out$error_bound,   # rigorous bound: (b-a)/2
    iterations   = out$iterations)
})

# Newton–Raphson results
res_newton <- lapply(seq_len(n_roots), function(i){
  cat(sprintf("\n[Newton–Raphson %d/%d]\n", i, n_roots))
  x0  <- get_newton_start_suggested(f, df, newton_suggestions[i])
  out <- newton_raphson(f, df, x0, tol = tol, maxit = maxit)
  c(method       = "Newton",
    root_id      = i,
    init         = sprintf("x0=%.4f", x0),
    estimate     = out$root,
    residual     = out$residual,
    err_estimate = out$step_last,     # practical proxy: |Δx_last|
    iterations   = out$iterations)
})


[Bisection 1/3]

Enter interval [a,b] for bisection (press Enter to accept the suggestion):
Suggested: [a, b] = [-2.0000, -1.0000]
Interval accepted: [-2.0000, -1.0000]

[Bisection 2/3]

Enter interval [a,b] for bisection (press Enter to accept the suggestion):
Suggested: [a, b] = [1.0000, 2.0000]
Interval accepted: [1.0000, 2.0000]

[Bisection 3/3]

Enter interval [a,b] for bisection (press Enter to accept the suggestion):
Suggested: [a, b] = [2.5000, 3.5000]
Interval accepted: [2.5000, 3.5000]

[Newton–Raphson 1/3]

Enter initial guess for Newton–Raphson (press Enter to accept the suggestion):
Suggested: x0 = -1.5000
Starting Newton from x0 = -1.5000 (f(x0)=-1.1250, f'(x0)=13.7500)

[Newton–Raphson 2/3]

Enter initial guess for Newton–Raphson (press Enter to accept the suggestion):
Suggested: x0 = 1.5000
Starting Newton from x0 = 1.5000 (f(x0)=-0.3750, f'(x0)=-4.2500)

[Newton–Raphson 3/3]

Enter initial guess for Newton–Raphson (press Enter to accept the suggestion):
Suggested: x0 

In [20]:
# Build results table
to_df <- function(lst){
  df <- as.data.frame(do.call(rbind, lst), stringsAsFactors = FALSE)
  # Type conversions
  num_cols <- c("estimate","residual","err_estimate","iterations")
  df[ , num_cols] <- lapply(df[ , num_cols, drop = FALSE], as.numeric)
  # Normalize types and order
  df$root_id <- as.integer(df$root_id)
  df <- df[order(df$method, df$root_id), ]
  rownames(df) <- NULL
  df
}

results <- to_df(c(res_bis, res_newton))

In [21]:
# Compare methods root-by-root

# Keep only the columns needed for the merge
bis_sub   <- results[results$method == "Bisection", c("root_id","estimate")]
newt_sub  <- results[results$method == "Newton",    c("root_id","estimate")]

# Merge by root_id and compute agreement diagnostics
cmp <- merge(bis_sub, newt_sub, by = "root_id", suffixes = c("_bis","_newton"))
cmp$abs_diff   <- abs(cmp$estimate_bis - cmp$estimate_newton)
cmp$within_tol <- cmp$abs_diff < tol

# Analytical cross-check (validation only)
# Exact roots for this polynomial: (-sqrt(2), +sqrt(2), 3)
roots_exact        <- c(-sqrt(2), sqrt(2), 3)
cmp$exact          <- roots_exact[cmp$root_id]
cmp$err_bis        <- abs(cmp$estimate_bis    - cmp$exact)
cmp$err_newton     <- abs(cmp$estimate_newton - cmp$exact)

In [22]:
label_df    <- data.frame(root_id = 1:3)

# Results table with display rounding
results_display <- merge(results, label_df, by = "root_id", all.x = TRUE)
results_display <- results_display[, c("root_id","method","init",
                                       "estimate","residual","err_estimate","iterations")]
results_display$estimate     <- round(results_display$estimate, 8)
results_display$residual     <- signif(results_display$residual, 3)
results_display$err_estimate <- signif(results_display$err_estimate, 3)

# Side-by-side comparison table
cmp_wide <- merge(label_df, cmp, by = "root_id", all.y = TRUE)
cmp_wide <- cmp_wide[, c("root_id", "estimate_bis",
                         "estimate_newton", "abs_diff",
                         "within_tol","err_bis","err_newton")]

# Display rounding for notebook
round_cols <- c("estimate_bis","estimate_newton","abs_diff","err_bis","err_newton")
cmp_wide_display <- cmp_wide
#cmp_wide_display[round_cols] <- lapply(cmp_wide_display[round_cols], function(x) round(x, 6))

In [23]:
# Safe pretty table printer
safe_kable <- function(df, caption = NULL) {
  if (requireNamespace("knitr", quietly = TRUE)) {
    print(knitr::kable(df, caption = caption, align = "l", format = "simple"))
  } else {
    if (!is.null(caption)) cat("\n", caption, "\n", sep = "")
    print(df, row.names = FALSE)
  }
}

In [24]:
cmp_wide_display$abs_diff   <- sprintf("%.6e", cmp_wide_display$abs_diff)
cmp_wide_display$err_bis    <- sprintf("%.6e", cmp_wide_display$err_bis)
cmp_wide_display$err_newton <- sprintf("%.6e", cmp_wide_display$err_newton)

In [25]:
cat("=== Results ===\n")
safe_kable(results_display, caption = "Runs for both methods")

cat("\n\n=== Method comparison per root (Bisection vs Newton) ===\n")
safe_kable(cmp_wide_display, caption = "Side-by-side comparison by root. 'within_tol' indicates agreement within tolerance.")

=== Results ===


Table: Runs for both methods

root_id   method      init                 estimate    residual   err_estimate   iterations 
--------  ----------  -------------------  ----------  ---------  -------------  -----------
1         Bisection   [-2.0000, -1.0000]   -1.414208   7.62e-05   7.60e-06       17         
1         Newton      x0=-1.5000           -1.414214   0.00e+00   9.10e-06       3          
2         Bisection   [1.0000, 2.0000]     1.414215    6.80e-06   3.05e-05       15         
2         Newton      x0=1.5000            1.414212    7.40e-06   2.45e-03       2          
3         Bisection   [2.5000, 3.5000]     3.000000    0.00e+00   5.00e-01       1          
3         Newton      x0=3.1000            3.000000    0.00e+00   4.81e-05       3          


=== Method comparison per root (Bisection vs Newton) ===


Table: Side-by-side comparison by root. 'within_tol' indicates agreement within tolerance.

root_id   estimate_bis   estimate_newton   abs_diff    

### Discussion of Results

Both **Bisection** and **Newton–Raphson** methods successfully found the three real roots of $f(x) = x^3 - 3x^2 - 2x + 6 $:  
$x_1 \approx -1.4142$; $x_2 \approx +1.4142$; $x_3 = 3$.

- The two methods agree within the tolerance $10^{-5}$ for all roots (`within_tol = TRUE`).
- **Bisection** needed about 15–17 iterations (linear convergence), while **Newton–Raphson** converged in 3–4 steps (quadratic convergence).
- Residuals are much smaller for Newton (up to $10^{-16}$), showing higher numerical precision.
- The estimated roots match the analytical values $-\sqrt{2}, +\sqrt{2}, 3$ within machine accuracy.

**Conclusion:**  

Bisection is slower but always reliable; Newton–Raphson is much faster and highly accurate if the initial guess is good.

In [27]:
# Write the required plain-text file
# File formatting: fixed decimals for numeric fields; plain tab-delimited
fmt_num <- function(x) formatC(x, digits = 6, format = "e")
out_txt <- results
out_txt$estimate     <- fmt_num(out_txt$estimate)
out_txt$residual     <- fmt_num(out_txt$residual)
out_txt$err_estimate <- fmt_num(out_txt$err_estimate)
out_txt$iterations   <- as.integer(out_txt$iterations)

write.table(out_txt,
            file = "root_results.txt",
            sep = "\t", quote = FALSE, row.names = FALSE)

cat('\nFile "root_results.txt" written in the current working directory.\n')


File "root_results.txt" written in the current working directory.
